In [110]:
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import tarfile
import shutil
import tempfile
import time
import pickle
import matplotlib.pyplot as plt
from monai.apps import DecathlonDataset, TciaDataset
from monai.config import print_config
from monai.data import DataLoader, decollate_batch
from monai.handlers.utils import from_engine
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.networks.nets import SegResNet
from monai.transforms import (
    Activations,
    Activationsd,
    AsDiscrete,
    AsDiscreted,
    Compose,
    Invertd,
    LoadImaged,
    MapTransform,
    NormalizeIntensityd,
    Orientationd,
    RandFlipd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    RandSpatialCropd,
    Spacingd,
    EnsureTyped,
    EnsureChannelFirstd,
    ResampleToMatchd,
)
from monai.utils import set_determinism

import torch
import torch.nn.parallel
import torch.distributed as dist

print_config()

MONAI version: 1.2.0
Numpy version: 1.25.0
Pytorch version: 2.0.1+cpu
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: c:\Users\Cecilia\AppData\Local\pypoetry\Cache\virtualenvs\unsup-seg-gbm-nLgqUjZ0-py3.11\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 10.0.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.0.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
py

## Ejemplo Decatlon Dataset

In [49]:
class ConvertToMultiChannelBasedOnBratsClassesd(MapTransform):
    """
    Convert labels to multi channels based on brats classes:
    label 1 is necrosis
    label 2 is edema
    label 3 is activo
    The possible classes are N (necrosis), E (edema)
    and TA (active).

    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            result = []
            # label 1 necro
            result.append(d[key] == 1)
            # label 2 is ET
            result.append(d[key] == 2)
            # merge labels 3, 4 and 3 to construct activo
            result.append(torch.logical_or(d[key] == 3, d[key] == 4))

            d[key] = torch.stack(result, axis=0).float()
        return d

In [83]:
train_transform = Compose(
    [
        # load 4 Nifti images and stack them together
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys="image"),
        EnsureTyped(keys=["image", "label"]),
        ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.0, 1.0, 1.0),
            mode=("bilinear", "nearest"),
        ),
        RandSpatialCropd(keys=["image", "label"], roi_size=[224, 224, 144], random_size=False),
        RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
        RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=1),
        RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=2),
        NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
        RandScaleIntensityd(keys="image", factors=0.1, prob=1.0),
        RandShiftIntensityd(keys="image", offsets=0.1, prob=1.0),
    ]
)
val_transform = Compose(
    [
        LoadImaged(keys=["image", "label"],allow_missing_keys=True),
        EnsureChannelFirstd(keys="image"),
        EnsureTyped(keys=["image", "label"]),
        ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.0, 1.0, 1.0),
            mode=("bilinear", "nearest"),
        ),
        NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
    ]
)

In [84]:
# here we don't cache any data in case out of memory issue
root_dir="./Dataset"
train_ds = DecathlonDataset(
    root_dir=root_dir,
    task="Task01_BrainTumour",
    transform=train_transform,
    section="training",
    download=False,
    cache_rate=0.0,
    num_workers=4,
)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=4)
val_ds = DecathlonDataset(
    root_dir=root_dir,
    task="Task01_BrainTumour",
    transform=val_transform,
    section="validation",
    download=False,
    cache_rate=0.0,
    num_workers=4,
)
val_loader = DataLoader(val_ds, batch_size=2, shuffle=False, num_workers=4)

In [116]:
val_ds[0]['image'].shape

torch.Size([4, 240, 240, 155])

In [128]:
transform = Compose(
    [
        #LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image"]),
        #ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
        #Orientationd(keys=["image"], axcodes="RAS"),
        #Spacingd(keys=["image"], pixdim=(1.0, 1.0, 1.0), mode="bilinear"),
        #NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
    ]
)

In [131]:
from src.get_data import CustomDataset

val=CustomDataset('./Dataset/Custom_dataset_1/', section="valid", transform=None)

Found 2 images and 2 labels.
Image files: [['./Dataset/Custom_dataset_1/valid\\images\\images_DSC\\UPENN-GBM-00011_11\\UPENN-GBM-00011_11_DSC_ap-rCBV.nii.gz', './Dataset/Custom_dataset_1/valid\\images\\images_DSC\\UPENN-GBM-00011_11\\UPENN-GBM-00011_11_DSC_PH.nii.gz', './Dataset/Custom_dataset_1/valid\\images\\images_DSC\\UPENN-GBM-00011_11\\UPENN-GBM-00011_11_DSC_PSR.nii.gz'], ['./Dataset/Custom_dataset_1/valid\\images\\images_DSC\\UPENN-GBM-00012_11\\UPENN-GBM-00012_11_DSC_ap-rCBV.nii.gz', './Dataset/Custom_dataset_1/valid\\images\\images_DSC\\UPENN-GBM-00012_11\\UPENN-GBM-00012_11_DSC_PH.nii.gz', './Dataset/Custom_dataset_1/valid\\images\\images_DSC\\UPENN-GBM-00012_11\\UPENN-GBM-00012_11_DSC_PSR.nii.gz']]
Label files: ['./Dataset/Custom_dataset_1/valid\\labels\\UPENN-GBM-00011_11_automated_approx_segm.nii.gz', './Dataset/Custom_dataset_1/valid\\labels\\UPENN-GBM-00012_11_automated_approx_segm.nii.gz']


In [133]:
val[0]['image'].shape

torch.Size([3, 240, 240, 155])

In [7]:
import os
from monai.data import Dataset, DataLoader
from monai.transforms import LoadImaged, Compose
from src.get_data import CustomDataset

In [8]:
transform = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image"]),
        #ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
        Orientationd(keys=["image"], axcodes="RAS"),
        Spacingd(keys=["image"], pixdim=(1.0, 1.0, 1.0), mode="bilinear"),
        NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
    ]
)

In [11]:
# Define your data transformations using MONAI transforms.
# Example: You can add more transforms based on your requirements.
#transform = Compose([YourOtherTransformsHere])

# Create instances of your custom dataset for training and validation.
train_dataset = CustomDataset(root_dir='E:\\Download\\UPENN-GBM\\Custom_dataset\\train', transform=transform)
#valid_dataset = CustomDataset(root_dir='path/to/valid', transform=transform)

# Create data loaders using the datasets.
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4)
#valid_loader = DataLoader(valid_dataset, batch_size=your_batch_size, shuffle=False, num_workers=your_num_workers)

# Access images and labels from the dataset
#sample = train_dataset[0]
#print(sample["image"], sample["label"])

Found 110 images and 10 labels.


In [18]:
train_dataset.label_files[0]

TypeError: 'DataLoader' object is not subscriptable